In [1]:
from langchain_community.llms import Ollama
from crewai import Agent, Task, Crew, Process

from langchain.agents import Tool
from langchain_experimental.utilities import PythonREPL

In [2]:
llm = Ollama(model="mistral")
response = llm.invoke("Write a python code that reads in a csv as a dataframe.")
print(response)

 To read a CSV file into a DataFrame using Python, you can use the `pandas` library. If you haven't installed it yet, you can do so by running `pip install pandas` in your terminal or command prompt. Here is an example of how to read a CSV file named "data.csv" into a DataFrame:

```python
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv('data.csv')

# Display the first five rows of the DataFrame to check if it was read correctly
print(df.head())
```

Replace "data.csv" with the name and path of your actual CSV file. If your CSV file is located in a different directory or has a different name, update the file path accordingly. The `head()` function is used to display the first five rows of the DataFrame for verification purposes.


In [3]:
python_tool = Tool(
    name="python_tool",
    description="Python shell that executed Python commands.",
    func=PythonREPL().run
)

agent = Agent(
    role='Data Cleaner',
    goal='Import and clean data',
    backstory='An experienced data analyst that is highly skilled in Python.',
    llm = llm,
    vebose=True,
    tools=[python_tool]
)

task = Task(
    agent=agent,
    description='Write a code that can read in a csv as a dataframe in Python.'
)

task.execute()

'To read a CSV file into a DataFrame using pandas library in Python, you can use the following code:\n\n```python\nimport pandas as pd\n\nAction: None'

In [2]:
from openai import OpenAI
from pydantic import BaseModel, Field
from typing import List

import instructor


class PythonCode(BaseModel):
    function_name: str
    code: str = Field(description="Provide python code that fulfills the user's request")


# enables `response_model` in create call
client = instructor.patch(
    OpenAI(
        base_url="http://localhost:11434/v1", #base_url of Ollama, v1 is for OpenAI I think
        api_key="ollama",  # required, but unused
    ),
    mode=instructor.Mode.JSON,
)

resp = client.chat.completions.create(
    model="llama2",
    messages=[
        {
            "role": "user",
            "content": "Write a python code to read in a csv as a dataframe",
        }
    ],
    response_model=PythonCode,
)
print(resp.model_dump_json(indent=2))

{
  "function_name": "Function Name",
  "code": "import pandas as pd \n df = pd.read_csv('data.csv')"
}


In [4]:
print(resp.code)

import pandas as pd 
 df = pd.read_csv('data.csv')
